In [31]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import xgboost as xgb

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, label_binarize
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [33]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [34]:
def create_convnext_like_feature_extractor(input_dim, sliding_window_size=3):
    input_layer = Input(shape=(input_dim,))
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)
    x = reshaped_input

    # Simulated ConvNeXt block
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 8)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)

    # Flatten the output for feature extraction
    feature_output = Flatten()(x)

    # Reconstruction head for self-supervised learning
    reconstruction_output = Dense(input_dim, activation='linear')(feature_output)

    # Create the model
    model = Model(inputs=input_layer, outputs=reconstruction_output)
    return model

In [35]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    train_X = train_data.drop(['label'], axis=1)
    if (is_string_labels):
        train_y = train_data['label'].map(label_mapping)
    else:
        train_y = train_data['label'] - 1
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    test_X = test_data.drop(['label'], axis=1)
    if (is_string_labels):
        test_y = test_data['label'].map(label_mapping)
    else:
        test_y = test_data['label'] - 1
    del test_data
    gc.collect()

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Define XGBoost parameters
    params = {
        "objective": "multi:softprob",
        "num_class": len(train_y.unique()),
        "eval_metric": "mlogloss",
        "max_depth": 6,
        "eta": 0.3,
        "seed": 42
    }

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs} started...")
                
        # Train the NN feature extractor
        feature_extractor = create_convnext_like_feature_extractor(train_X.shape[1])
        feature_extractor.compile(optimizer='adam', loss='mean_squared_error')
        feature_extractor.fit(train_X, train_X, epochs=3, batch_size=32, verbose=0)  # Self-supervised training

        # Extract features
        feature_extractor_truncated = Model(inputs=feature_extractor.input, outputs=feature_extractor.layers[-2].output)
        train_features = feature_extractor_truncated.predict(train_X, verbose=0)
        test_features = feature_extractor_truncated.predict(test_X, verbose=0)

        # Convert data to DMatrix
        dtrain = xgb.DMatrix(data=train_features, label=train_y)
        dtest = xgb.DMatrix(data=test_features, label=test_y)

        # Train XGBoost
        num_round = 100
        bst = xgb.train(params, dtrain, num_round)

        # Make predictions
        test_probabilities = bst.predict(dtest)
        test_predictions = np.argmax(test_probabilities, axis=1)
        del feature_extractor
        gc.collect()

        test_y_binarized = label_binarize(test_y, classes=range(len(np.unique(test_y))))

        # Compute metrics
        accuracy = accuracy_score(test_y, test_predictions)
        balanced_acc = balanced_accuracy_score(test_y, test_predictions)
        roc_auc = roc_auc_score(test_y, test_probabilities, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc_scores = []
        for i in range(test_y_binarized.shape[1]):  # Iterate over each class
            pr_auc = average_precision_score(test_y_binarized[:, i], test_probabilities[:, i])
            pr_auc_scores.append(pr_auc)
        pr_auc = np.mean(pr_auc_scores)
        f2 = fbeta_score(test_y, test_predictions, beta=2, average='weighted')
        mcc = matthews_corrcoef(test_y, test_predictions)

        # Store metrics
        metrics_storage['test_accuracy'].append(accuracy)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(test_y, test_predictions, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()

In [ ]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/50 started...

Aggregated Metrics for 1 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9872
roc_auc: 1.0000
pr_auc: 0.9942
f2: 0.9996
mcc: 0.9988
0_precision: 0.9996
0_recall: 0.9999
0_f1-score: 0.9997
0_support: 9117.0000
1_precision: 1.0000
1_recall: 1.0000
1_f1-score: 1.0000
1_support: 10.0000
2_precision: 0.9688
2_recall: 0.9118
2_f1-score: 0.9394
2_support: 34.0000
3_precision: 1.0000
3_recall: 1.0000
3_f1-score: 1.0000
3_support: 1781.0000
4_precision: 1.0000
4_recall: 0.9985
4_f1-score: 0.9992
4_support: 653.0000
5_precision: 1.0000
5_recall: 1.0000
5_f1-score: 1.0000
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9996
macro avg_precision: 0.9955
macro avg_recall: 0.9872
macro avg_f1-score: 0.9912
macro avg_support: 11600.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0.9996
weighted avg_f1-score: 0.9996
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs:
test_accuracy: 0.

In [ ]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_accuracy: 0.8927
balanced_accuracy: 0.8560
roc_auc: 0.9898
pr_auc: 0.9470
f2: 0.8924
mcc: 0.8271
0_precision: 0.8958
0_recall: 0.8590
0_f1-score: 0.8770
0_support: 42368.0000
1_precision: 0.8857
1_recall: 0.9217
1_f1-score: 0.9034
1_support: 56661.0000
2_precision: 0.9064
2_recall: 0.9280
2_f1-score: 0.9171
2_support: 7151.0000
3_precision: 0.8865
3_recall: 0.8251
3_f1-score: 0.8547
3_support: 549.0000
4_precision: 0.9041
4_recall: 0.6951
4_f1-score: 0.7859
4_support: 1899.0000
5_precision: 0.8628
5_recall: 0.8313
5_f1-score: 0.8468
5_support: 3473.0000
6_precision: 0.9615
6_recall: 0.9317
6_f1-score: 0.9464
6_support: 4102.0000
accuracy: 0.8927
macro avg_precision: 0.9004
macro avg_recall: 0.8560
macro avg_f1-score: 0.8759
macro avg_support: 116203.0000
weighted avg_precision: 0.8930
weighted avg_recall: 0.8927
weighted avg_f1-score: 0.8923
weighted avg_support: 116203.0000
Run 2/50 started...



Aggregated Metrics for 2 runs:
test_accuracy: 0.8932
balanced_accuracy: 0.8563
roc_auc: 0.9900
pr_auc: 0.9471
f2: 0.8929
mcc: 0.8278
0_precision: 0.8965
0_recall: 0.8602
0_f1-score: 0.8780
0_support: 42368.0000
1_precision: 0.8862
1_recall: 0.9221
1_f1-score: 0.9038
1_support: 56661.0000
2_precision: 0.9096
2_recall: 0.9274
2_f1-score: 0.9184
2_support: 7151.0000
3_precision: 0.8778
3_recall: 0.8361
3_f1-score: 0.8563
3_support: 549.0000
4_precision: 0.8975
4_recall: 0.6869
4_f1-score: 0.7782
4_support: 1899.0000
5_precision: 0.8608
5_recall: 0.8287
5_f1-score: 0.8444
5_support: 3473.0000
6_precision: 0.9596
6_recall: 0.9325
6_f1-score: 0.9458
6_support: 4102.0000
accuracy: 0.8932
macro avg_precision: 0.8983
macro avg_recall: 0.8563
macro avg_f1-score: 0.8750
macro avg_support: 116203.0000
weighted avg_precision: 0.8934
weighted avg_recall: 0.8932
weighted avg_f1-score: 0.8927
weighted avg_support: 116203.0000
Run 3/50 started...



Aggregated Metrics for 3 runs:
test_accuracy: 0.8916
balanced_accuracy: 0.8570
roc_auc: 0.9899
pr_auc: 0.9468
f2: 0.8913
mcc: 0.8253
0_precision: 0.8942
0_recall: 0.8584
0_f1-score: 0.8759
0_support: 42368.0000
1_precision: 0.8851
1_recall: 0.9203
1_f1-score: 0.9024
1_support: 56661.0000
2_precision: 0.9090
2_recall: 0.9263
2_f1-score: 0.9176
2_support: 7151.0000
3_precision: 0.8800
3_recall: 0.8409
3_f1-score: 0.8600
3_support: 549.0000
4_precision: 0.8968
4_recall: 0.6974
4_f1-score: 0.7845
4_support: 1899.0000
5_precision: 0.8581
5_recall: 0.8240
5_f1-score: 0.8407
5_support: 3473.0000
6_precision: 0.9573
6_recall: 0.9314
6_f1-score: 0.9442
6_support: 4102.0000
accuracy: 0.8916
macro avg_precision: 0.8972
macro avg_recall: 0.8570
macro avg_f1-score: 0.8750
macro avg_support: 116203.0000
weighted avg_precision: 0.8918
weighted avg_recall: 0.8916
weighted avg_f1-score: 0.8912
weighted avg_support: 116203.0000
Run 4/50 started...



Aggregated Metrics for 4 runs:
test_accuracy: 0.8902
balanced_accuracy: 0.8547
roc_auc: 0.9896
pr_auc: 0.9457
f2: 0.8899
mcc: 0.8230
0_precision: 0.8928
0_recall: 0.8569
0_f1-score: 0.8745
0_support: 42368.0000
1_precision: 0.8836
1_recall: 0.9193
1_f1-score: 0.9011
1_support: 56661.0000
2_precision: 0.9071
2_recall: 0.9248
2_f1-score: 0.9159
2_support: 7151.0000
3_precision: 0.8781
3_recall: 0.8420
3_f1-score: 0.8596
3_support: 549.0000
4_precision: 0.8969
4_recall: 0.6904
4_f1-score: 0.7801
4_support: 1899.0000
5_precision: 0.8561
5_recall: 0.8202
5_f1-score: 0.8377
5_support: 3473.0000
6_precision: 0.9567
6_recall: 0.9295
6_f1-score: 0.9429
6_support: 4102.0000
accuracy: 0.8902
macro avg_precision: 0.8959
macro avg_recall: 0.8547
macro avg_f1-score: 0.8731
macro avg_support: 116203.0000
weighted avg_precision: 0.8904
weighted avg_recall: 0.8902
weighted avg_f1-score: 0.8897
weighted avg_support: 116203.0000
Run 5/50 started...


...


Aggregated Metrics for 19 runs:
test_accuracy: 0.8883
balanced_accuracy: 0.8508
roc_auc: 0.9893
pr_auc: 0.9430
f2: 0.8879
mcc: 0.8199
0_precision: 0.8914
0_recall: 0.8550
0_f1-score: 0.8728
0_support: 42368.0000
1_precision: 0.8817
1_recall: 0.9182
1_f1-score: 0.8996
1_support: 56661.0000
2_precision: 0.9039
2_recall: 0.9221
2_f1-score: 0.9129
2_support: 7151.0000
3_precision: 0.8801
3_recall: 0.8439
3_f1-score: 0.8616
3_support: 549.0000
4_precision: 0.8943
4_recall: 0.6764
4_f1-score: 0.7701
4_support: 1899.0000
5_precision: 0.8523
5_recall: 0.8124
5_f1-score: 0.8318
5_support: 3473.0000
6_precision: 0.9549
6_recall: 0.9277
6_f1-score: 0.9411
6_support: 4102.0000
accuracy: 0.8883
macro avg_precision: 0.8941
macro avg_recall: 0.8508
macro avg_f1-score: 0.8700
macro avg_support: 116203.0000
weighted avg_precision: 0.8885
weighted avg_recall: 0.8883
weighted avg_f1-score: 0.8878
weighted avg_support: 116203.0000
Run 20/50 started...



Aggregated Metrics for 20 runs:
test_accuracy: 0.8882
balanced_accuracy: 0.8507
roc_auc: 0.9893
pr_auc: 0.9429
f2: 0.8878
mcc: 0.8197
0_precision: 0.8912
0_recall: 0.8549
0_f1-score: 0.8727
0_support: 42368.0000
1_precision: 0.8816
1_recall: 0.9181
1_f1-score: 0.8995
1_support: 56661.0000
2_precision: 0.9037
2_recall: 0.9221
2_f1-score: 0.9128
2_support: 7151.0000
3_precision: 0.8799
3_recall: 0.8443
3_f1-score: 0.8616
3_support: 549.0000
4_precision: 0.8943
4_recall: 0.6762
4_f1-score: 0.7700
4_support: 1899.0000
5_precision: 0.8524
5_recall: 0.8120
5_f1-score: 0.8317
5_support: 3473.0000
6_precision: 0.9549
6_recall: 0.9272
6_f1-score: 0.9408
6_support: 4102.0000
accuracy: 0.8882
macro avg_precision: 0.8940
macro avg_recall: 0.8507
macro avg_f1-score: 0.8699
macro avg_support: 116203.0000
weighted avg_precision: 0.8884
weighted avg_recall: 0.8882
weighted avg_f1-score: 0.8877
weighted avg_support: 116203.0000
Run 21/50 started...



Aggregated Metrics for 21 runs:
test_accuracy: 0.8881
balanced_accuracy: 0.8506
roc_auc: 0.9893
pr_auc: 0.9429
f2: 0.8878
mcc: 0.8196
0_precision: 0.8912
0_recall: 0.8548
0_f1-score: 0.8726
0_support: 42368.0000
1_precision: 0.8815
1_recall: 0.9181
1_f1-score: 0.8994
1_support: 56661.0000
2_precision: 0.9036
2_recall: 0.9219
2_f1-score: 0.9127
2_support: 7151.0000
3_precision: 0.8802
3_recall: 0.8439
3_f1-score: 0.8616
3_support: 549.0000
4_precision: 0.8944
4_recall: 0.6765
4_f1-score: 0.7702
4_support: 1899.0000
5_precision: 0.8523
5_recall: 0.8121
5_f1-score: 0.8317
5_support: 3473.0000
6_precision: 0.9547
6_recall: 0.9272
6_f1-score: 0.9408
6_support: 4102.0000
accuracy: 0.8881
macro avg_precision: 0.8940
macro avg_recall: 0.8506
macro avg_f1-score: 0.8699
macro avg_support: 116203.0000
weighted avg_precision: 0.8883
weighted avg_recall: 0.8881
weighted avg_f1-score: 0.8876
weighted avg_support: 116203.0000
Run 22/50 started...



Aggregated Metrics for 22 runs:
test_accuracy: 0.8882
balanced_accuracy: 0.8507
roc_auc: 0.9893
pr_auc: 0.9430
f2: 0.8879
mcc: 0.8198
0_precision: 0.8913
0_recall: 0.8549
0_f1-score: 0.8727
0_support: 42368.0000
1_precision: 0.8816
1_recall: 0.9182
1_f1-score: 0.8996
1_support: 56661.0000
2_precision: 0.9036
2_recall: 0.9221
2_f1-score: 0.9128
2_support: 7151.0000
3_precision: 0.8807
3_recall: 0.8434
3_f1-score: 0.8616
3_support: 549.0000
4_precision: 0.8944
4_recall: 0.6763
4_f1-score: 0.7701
4_support: 1899.0000
5_precision: 0.8529
5_recall: 0.8126
5_f1-score: 0.8322
5_support: 3473.0000
6_precision: 0.9547
6_recall: 0.9274
6_f1-score: 0.9408
6_support: 4102.0000
accuracy: 0.8882
macro avg_precision: 0.8942
macro avg_recall: 0.8507
macro avg_f1-score: 0.8700
macro avg_support: 116203.0000
weighted avg_precision: 0.8884
weighted avg_recall: 0.8882
weighted avg_f1-score: 0.8877
weighted avg_support: 116203.0000
Run 23/50 started...



Aggregated Metrics for 23 runs:
test_accuracy: 0.8881
balanced_accuracy: 0.8503
roc_auc: 0.9893
pr_auc: 0.9428
f2: 0.8877
mcc: 0.8195
0_precision: 0.8911
0_recall: 0.8548
0_f1-score: 0.8726
0_support: 42368.0000
1_precision: 0.8815
1_recall: 0.9181
1_f1-score: 0.8994
1_support: 56661.0000
2_precision: 0.9036
2_recall: 0.9221
2_f1-score: 0.9127
2_support: 7151.0000
3_precision: 0.8808
3_recall: 0.8431
3_f1-score: 0.8615
3_support: 549.0000
4_precision: 0.8941
4_recall: 0.6745
4_f1-score: 0.7689
4_support: 1899.0000
5_precision: 0.8526
5_recall: 0.8123
5_f1-score: 0.8319
5_support: 3473.0000
6_precision: 0.9546
6_recall: 0.9274
6_f1-score: 0.9408
6_support: 4102.0000
accuracy: 0.8881
macro avg_precision: 0.8940
macro avg_recall: 0.8503
macro avg_f1-score: 0.8697
macro avg_support: 116203.0000
weighted avg_precision: 0.8883
weighted avg_recall: 0.8881
weighted avg_f1-score: 0.8876
weighted avg_support: 116203.0000
Run 24/50 started...


In [ ]:
run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = False)

In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9816
roc_auc: 1.0000
pr_auc: 0.9978
f2: 0.9996
mcc: 0.9985
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9996
1_recall: 1.0000
1_f1-score: 0.9998
1_support: 2657.0000
2_precision: 1.0000
2_recall: 0.9967
2_f1-score: 0.9983
2_support: 908.0000
3_precision: 1.0000
3_recall: 1.0000
3_f1-score: 1.0000
3_support: 522.0000
4_precision: 0.9932
4_recall: 0.9966
4_f1-score: 0.9949
4_support: 293.0000
5_precision: 0.9963
5_recall: 1.0000
5_f1-score: 0.9981
5_support: 269.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 116.0000
7_precision: 0.9400
7_recall: 0.8868
7_f1-score: 0.9126
7_support: 53.0000
8_precision: 0.9767
8_recall: 0.9545
8_f1-score: 0.9655
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9895
macro avg_recall: 0.9816
macro avg_f1-score: 0.9855
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0.


Aggregated Metrics for 2 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9806
roc_auc: 1.0000
pr_auc: 0.9970
f2: 0.9996
mcc: 0.9985
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 1.0000
1_f1-score: 0.9999
1_support: 2657.0000
2_precision: 1.0000
2_recall: 0.9978
2_f1-score: 0.9989
2_support: 908.0000
3_precision: 1.0000
3_recall: 0.9990
3_f1-score: 0.9995
3_support: 522.0000
4_precision: 0.9932
4_recall: 0.9949
4_f1-score: 0.9940
4_support: 293.0000
5_precision: 0.9944
5_recall: 0.9981
5_f1-score: 0.9963
5_support: 269.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 116.0000
7_precision: 0.9589
7_recall: 0.8585
7_f1-score: 0.9053
7_support: 53.0000
8_precision: 0.9565
8_recall: 0.9773
8_f1-score: 0.9663
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9892
macro avg_recall: 0.9806
macro avg_f1-score: 0.9845
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0.


Aggregated Metrics for 3 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9800
roc_auc: 1.0000
pr_auc: 0.9968
f2: 0.9995
mcc: 0.9984
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9999
1_f1-score: 0.9999
1_support: 2657.0000
2_precision: 0.9989
2_recall: 0.9978
2_f1-score: 0.9983
2_support: 908.0000
3_precision: 1.0000
3_recall: 0.9981
3_f1-score: 0.9990
3_support: 522.0000
4_precision: 0.9932
4_recall: 0.9943
4_f1-score: 0.9937
4_support: 293.0000
5_precision: 0.9951
5_recall: 0.9988
5_f1-score: 0.9969
5_support: 269.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 116.0000
7_precision: 0.9459
7_recall: 0.8616
7_f1-score: 0.9013
7_support: 53.0000
8_precision: 0.9632
8_recall: 0.9697
8_f1-score: 0.9660
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9884
macro avg_recall: 0.9800
macro avg_f1-score: 0.9839
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.


Aggregated Metrics for 4 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9804
roc_auc: 1.0000
pr_auc: 0.9969
f2: 0.9996
mcc: 0.9984
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9998
1_f1-score: 0.9999
1_support: 2657.0000
2_precision: 0.9986
2_recall: 0.9981
2_f1-score: 0.9983
2_support: 908.0000
3_precision: 1.0000
3_recall: 0.9981
3_f1-score: 0.9990
3_support: 522.0000
4_precision: 0.9940
4_recall: 0.9949
4_f1-score: 0.9945
4_support: 293.0000
5_precision: 0.9954
5_recall: 0.9991
5_f1-score: 0.9972
5_support: 269.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 116.0000
7_precision: 0.9492
7_recall: 0.8679
7_f1-score: 0.9063
7_support: 53.0000
8_precision: 0.9666
8_recall: 0.9659
8_f1-score: 0.9659
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9893
macro avg_recall: 0.9804
macro avg_f1-score: 0.9846
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0.

...


Aggregated Metrics for 27 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9813
roc_auc: 1.0000
pr_auc: 0.9972
f2: 0.9996
mcc: 0.9984
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9998
1_f1-score: 0.9998
1_support: 2657.0000
2_precision: 0.9985
2_recall: 0.9978
2_f1-score: 0.9982
2_support: 908.0000
3_precision: 0.9990
3_recall: 0.9975
3_f1-score: 0.9983
3_support: 522.0000
4_precision: 0.9948
4_recall: 0.9965
4_f1-score: 0.9956
4_support: 293.0000
5_precision: 0.9958
5_recall: 0.9985
5_f1-score: 0.9971
5_support: 269.0000
6_precision: 0.9981
6_recall: 0.9965
6_f1-score: 0.9973
6_support: 116.0000
7_precision: 0.9537
7_recall: 0.8721
7_f1-score: 0.9108
7_support: 53.0000
8_precision: 0.9701
8_recall: 0.9731
8_f1-score: 0.9714
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9900
macro avg_recall: 0.9813
macro avg_f1-score: 0.9854
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0


Aggregated Metrics for 28 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9811
roc_auc: 1.0000
pr_auc: 0.9972
f2: 0.9996
mcc: 0.9984
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9998
1_f1-score: 0.9998
1_support: 2657.0000
2_precision: 0.9985
2_recall: 0.9979
2_f1-score: 0.9982
2_support: 908.0000
3_precision: 0.9990
3_recall: 0.9975
3_f1-score: 0.9983
3_support: 522.0000
4_precision: 0.9949
4_recall: 0.9963
4_f1-score: 0.9956
4_support: 293.0000
5_precision: 0.9958
5_recall: 0.9984
5_f1-score: 0.9971
5_support: 269.0000
6_precision: 0.9982
6_recall: 0.9966
6_f1-score: 0.9974
6_support: 116.0000
7_precision: 0.9531
7_recall: 0.8706
7_f1-score: 0.9097
7_support: 53.0000
8_precision: 0.9688
8_recall: 0.9724
8_f1-score: 0.9704
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9898
macro avg_recall: 0.9811
macro avg_f1-score: 0.9852
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0


Aggregated Metrics for 29 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9808
roc_auc: 1.0000
pr_auc: 0.9972
f2: 0.9996
mcc: 0.9984
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9998
1_f1-score: 0.9998
1_support: 2657.0000
2_precision: 0.9985
2_recall: 0.9979
2_f1-score: 0.9982
2_support: 908.0000
3_precision: 0.9989
3_recall: 0.9976
3_f1-score: 0.9983
3_support: 522.0000
4_precision: 0.9948
4_recall: 0.9962
4_f1-score: 0.9955
4_support: 293.0000
5_precision: 0.9958
5_recall: 0.9985
5_f1-score: 0.9971
5_support: 269.0000
6_precision: 0.9982
6_recall: 0.9961
6_f1-score: 0.9972
6_support: 116.0000
7_precision: 0.9518
7_recall: 0.8692
7_f1-score: 0.9084
7_support: 53.0000
8_precision: 0.9683
8_recall: 0.9718
8_f1-score: 0.9699
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9896
macro avg_recall: 0.9808
macro avg_f1-score: 0.9849
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0


Aggregated Metrics for 30 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9809
roc_auc: 1.0000
pr_auc: 0.9972
f2: 0.9996
mcc: 0.9984
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9998
1_f1-score: 0.9998
1_support: 2657.0000
2_precision: 0.9985
2_recall: 0.9978
2_f1-score: 0.9982
2_support: 908.0000
3_precision: 0.9990
3_recall: 0.9975
3_f1-score: 0.9982
3_support: 522.0000
4_precision: 0.9947
4_recall: 0.9961
4_f1-score: 0.9954
4_support: 293.0000
5_precision: 0.9958
5_recall: 0.9985
5_f1-score: 0.9972
5_support: 269.0000
6_precision: 0.9977
6_recall: 0.9963
6_f1-score: 0.9970
6_support: 116.0000
7_precision: 0.9514
7_recall: 0.8692
7_f1-score: 0.9081
7_support: 53.0000
8_precision: 0.9686
8_recall: 0.9727
8_f1-score: 0.9705
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9895
macro avg_recall: 0.9809
macro avg_f1-score: 0.9849
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0


Aggregated Metrics for 31 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9810
roc_auc: 1.0000
pr_auc: 0.9972
f2: 0.9996
mcc: 0.9984
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9998
1_f1-score: 0.9998
1_support: 2657.0000
2_precision: 0.9985
2_recall: 0.9978
2_f1-score: 0.9982
2_support: 908.0000
3_precision: 0.9990
3_recall: 0.9975
3_f1-score: 0.9982
3_support: 522.0000
4_precision: 0.9948
4_recall: 0.9960
4_f1-score: 0.9954
4_support: 293.0000
5_precision: 0.9957
5_recall: 0.9986
5_f1-score: 0.9971
5_support: 269.0000
6_precision: 0.9978
6_recall: 0.9964
6_f1-score: 0.9971
6_support: 116.0000
7_precision: 0.9505
7_recall: 0.8704
7_f1-score: 0.9083
7_support: 53.0000
8_precision: 0.9689
8_recall: 0.9721
8_f1-score: 0.9703
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9894
macro avg_recall: 0.9810
macro avg_f1-score: 0.9849
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0